In [ ]:
# [예제 4.1] ‘2018년 2차_졸업생의 진로 현황(전체)’ 엑셀 파일을 판다스의 read_excel로 읽어오기 

import pandas as pd

# graduate_path = r'C:\Users\JK\Desktop\부동산 데이터\2018년 2차_졸업생의 진로 현황(전체).xlsx'
graduate_path = r' 졸업생 진로 데이터 엑셀 파일 디렉터리 \ 졸업생 진로 데이터 엑셀 파일명.xlsx'
raw_graduate = pd.read_excel(graduate_path, sheet_name='2018_졸업생의 진로 현황(중)')

In [ ]:
# [예제 4.2] 전국 중학교 졸업자 데이터를 데이터프레임으로 정리 

select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
graduate_data = select_col.drop(0)
graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']

In [ ]:
# [예제 4.3] 전국 중학교 지역명 정리

def get_sido(x):
    temp = x.split(' ')[0]
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0] + temp[2]

graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])

In [ ]:
# [예제 4.4] 누락된 지역 예외 처리

graduate_data.at[588,'시도'] = '부산'
graduate_data.at[588,'구군'] = '기장군'
graduate_data.at[3011,'시도'] = '경북'
graduate_data.at[3011,'구군'] = '예천군'

In [ ]:
# [예제 4.5] 특목고 진학 데이터를 데이터프레임으로 만드는 함수 

def graduate_preprocrssing(path):
    raw_graduate = pd.read_excel(path, sheet_name='2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
    graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
    graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
    
    def get_sido(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])
    
    graduate_data.at[588,'시도'] = '부산'
    graduate_data.at[588,'구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data

In [ ]:
# [예제 4.6] 평균 아파트 매매가격 데이터 읽어오기

# price_path = r'C:\Users\JK\Desktop\부동산 데이터\평균매매가격_아파트.xlsx'
price_path = r' 평균 아파트 매매가격 데이터 디렉터리 \ 평균 아파트 매매가격 데이터 엑셀 파일명.xlsx'
row_price = pd.read_excel(price_path, skiprows=10)

In [ ]:
# [예제 4.7] 지역 설정하기

big_col = []
for num, temp in enumerate(row_price['지 역']):
    if pd.isna(temp) :
        big_col.append(big_col[num-1])
    else:
        big_col.append(temp)
    
    
small_col = []
for num in range(len(row_price)):
    temp_list = list(row_price[['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']].iloc[num])
    for temp in temp_list[3::-1]:
        if not pd.isna(temp):
            small_col.append(temp)
            break
            
row_price.index = [big_col, small_col]

In [ ]:
# [예제 4.8] 필요 없는 컬럼을 없애고 컬럼과 인덱스 바꾸기 

transposed_price = row_price.drop(['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], axis=1).T

In [ ]:
# [예제 4.9] 날짜 인덱스 개선

time_index = []
for time in transposed_price.index:
    temp = time.split(' ')
    time_index.append(temp[0][:-1]+'.'+temp[1][:-1])

transposed_price.index = pd.to_datetime(time_index)

In [ ]:
# [예제 4.10] 평균 아파트 매매가 데이터를 데이터프레임으로 만드는 함수 

def gamjungwon_price_preprocessing(path):
    row_price = pd.read_excel(path, skiprows=10)
    
    big_col = []
    for num, temp in enumerate(row_price['지 역']):
        if pd.isna(temp) :
            big_col.append(big_col[num-1])
        else:
            big_col.append(temp)


    small_col = []
    for num in range(len(row_price)):
        temp_list = list(row_price[['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']].iloc[num])
        for temp in temp_list[3::-1]:
            if not pd.isna(temp):
                small_col.append(temp)
                break

    row_price.index = [big_col, small_col]
    
    transposed_price = row_price.drop(['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], axis=1).T
    
    time_index = []
    for time in transposed_price.index:
        temp = time.split(' ')
        time_index.append(temp[0][:-1]+'.'+temp[1][:-1])

    transposed_price.index = pd.to_datetime(time_index)
    
    return transposed_price

In [ ]:
# [예제 4.11] 특목고 데이터프레임을 이용해 시도별 졸업생의 데이터를 합산하기 

graduate_path = r' 졸업생 진로 데이터 엑셀 파일 디렉터리 \ 졸업생 진로 데이터 엑셀 파일명.xlsx'
gradu_df = graduate_preprocrssing(graduate_path)

gradu_df.groupby('시도').sum()

In [ ]:
# [예제 4.12] 정렬을 이용해 특목고를 가장 많이 진학시킨 시도 찾기 

gradu_sido = gradu_df.groupby('시도').sum()
gradu_sido.sort_values(by='총합', ascending=False)

In [ ]:
# [예제 4.13] 특목고 진학률을 계산해 정렬하기 

gradu_sido['진학률'] = gradu_sido['총합'] / gradu_sido['졸업자'] * 100
gradu_sido.sort_values(by='진학률', ascending=False)

In [ ]:
# [예제 4.14] 지역별 가격 가져와 저장하기

price_path = r' 평균 아파트 매매가격 데이터 디렉터리 \ 평균 아파트 매매가격 데이터 엑셀 파일명.xlsx'
price_data = gamjungwon_price_preprocessing(price_path)

sido_list = []
for i in gradu_sido.index:
    sido_list.append(price_data.loc['2018-6-1'][i][i])
      
gradu_sido['평균매매가격'] = sido_list

In [ ]:
# [예제 4.15] 그래프를 그리기 위한 설정

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline


font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# [예제 4.16] 특목고 진학률과 평균 아파트 매매가격의 scatter 그래프 그리기

plt.figure(figsize=(10, 7))
plt.scatter(gradu_sido['진학률'], gradu_sido['평균매매가격'], color='darkcyan', s=50)
plt.xlabel('졸업생 대비 특목고 진학생 비율(%)')
plt.ylabel('평균아파트매매가격')
for name in gradu_sido.index:
    plt.text(gradu_sido['진학률'][name]*1.02, gradu_sido['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
# [예제 4.17] 선형 회귀선 추가하기
import seaborn as sns

plt.figure(figsize=(10, 7))
plt.scatter(gradu_sido['진학률'], gradu_sido['평균매매가격'], color='darkcyan', s=50)
sns.regplot(gradu_sido['진학률'], gradu_sido['평균매매가격'], scatter=False, color='darkcyan')
plt.xlabel('졸업생 대비 특목고 진학생 비율(%)')
plt.ylabel('평균아파트매매가격')
for name in gradu_sido.index:
    plt.text(gradu_sido['진학률'][name]*1.02, gradu_sido['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
#[예제 4.18] 서울시의 구별 특목고 진학자 알아보기

local = '서울'

gradu_gu = graduate_data[graduate_data['시도'] == local].groupby('구군').sum()
gradu_gu['진학률'] = gradu_gu['총합'] / gradu_gu['졸업자'] * 100
gradu_gu['평균매매가격'] = price_data.loc['2018-6-1'][local][gradu_gu.index]

In [ ]:
# [예제 4.19] 서울시의 특목고 진학률과 평균 아파트 매매가격 scatter 그래프 그리기 

plt.figure(figsize=(12, 7))
plt.scatter(gradu_gu['진학률'], gradu_gu['평균매매가격'], color='steelblue', s=50)
sns.regplot(gradu_gu['진학률'], gradu_gu['평균매매가격'], scatter=False, color='steelblue')
plt.xlabel('졸업생 대비 특목고 진학생 비율(%)')
plt.ylabel('평균아파트매매가격')
for name in gradu_gu.index:
    plt.text(gradu_gu['진학률'][name]*1.02, gradu_gu['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
# [예제 4.20] 부산시의 특목고 진학률과 평균 아파트 매매가격 scatter 그래프 그리기 

local = '부산'

gradu_gu = graduate_data[graduate_data['시도'] == local].groupby('구군').sum()
gradu_gu['진학률'] = gradu_gu['총합'] / gradu_gu['졸업자'] * 100
gradu_gu['평균매매가격'] = price_data.loc['2018-6-1'][local][gradu_gu.index]
gradu_gu = gradu_gu.dropna()

plt.figure(figsize=(12, 7))
plt.scatter(gradu_gu['진학률'], gradu_gu['평균매매가격'], color='steelblue', s=50)
sns.regplot(gradu_gu['진학률'], gradu_gu['평균매매가격'], scatter=False, color='steelblue')
plt.xlabel('졸업생 대비 특목고 진학생 비율(%)')
plt.ylabel('평균아파트매매가격')
for name in gradu_gu.index:
    plt.text(gradu_gu['진학률'][name]*1.02, gradu_gu['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
# [예제 4.21] 학원 데이터를 데이터프레임으로 가져오기

#aca_path = r'C:\Users\JK\Desktop\부동산 데이터\2018년 시도별 행정구역별 사설학원 현황.xlsx'
aca_path = r' 사설학원 현황 데이터 디렉터리 \ 사설학원 현황 엑셀 파일명.xlsx'
aca_raw = pd.read_excel(aca_path , skiprows=3)

In [ ]:
# [예제 4.22] 학원 데이터프레임을 정리 

aca_data = aca_raw[aca_raw['분야'] == '입시검정및보습']
aca_data = aca_data[['시도', '행정구역', '학원수', '정원\n(수강자수)', '강사수', '강의실수','월평균교습시간','월평균교습비(원)' ]]
aca_data.columns = ['시도', '구군', '학원수', '수강자수', '강사수', '강의실수','월평균교습시간','월평균교습비']

In [ ]:
# [예제 4.23] 전국 시도의 학원수와 평균 아파트 매매가격의 scatter 그래프 그리기 

aca_sido = aca_data.groupby('시도').sum()

sido_list = []
for i in aca_sido.index:
    sido_list.append(price_data.loc['2018-6-1'][i][i])
      
aca_sido['평균매매가격'] = sido_list

plt.figure(figsize=(10, 7))
plt.scatter(aca_sido['학원수'], aca_sido['평균매매가격'], color='orange', s=50)
sns.regplot(aca_sido['학원수'], aca_sido['평균매매가격'], scatter=False, color='orange')
plt.xlabel('학원수')
plt.ylabel('평균아파트매매가격')
for name in aca_sido.index:
    plt.text(aca_sido['학원수'][name]*1.02, aca_sido['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
# [예제 4.24] 서울시의 학원 수와 평균 아파트 매매가격의 scatter 그래프 그리기 

local = '서울'

aca_gu = aca_data[aca_data['시도'] == local].groupby('구군').sum()
aca_gu['평균매매가격'] = price_data.loc['2018-6-1'][local][aca_gu.index]
aca_gu = aca_gu.dropna()

plt.figure(figsize=(12, 7))
plt.scatter(aca_gu['학원수'], aca_gu['평균매매가격'], color='orange', s=50)
sns.regplot(aca_gu['학원수'], aca_gu['평균매매가격'], scatter=False, color='orange')
plt.xlabel('학원수')
plt.ylabel('평균아파트매매가격')
for name in aca_gu.index:
    plt.text(aca_gu['학원수'][name]*1.02, aca_gu['평균매매가격'][name], name, fontsize=13)
        
plt.show()

In [ ]:
# [예제 4.25] 서울시의 특목고 진학률과 학원수를 하나의 데이터프레임으로 병합 

local = '서울'

gradu_gu = graduate_data[graduate_data['시도'] == local].groupby('구군').sum()
gradu_gu['진학률'] = gradu_gu['총합'] / gradu_gu['졸업자'] * 100
aca_gu = aca_data[aca_data['시도'] == local].groupby('구군').sum()
study_df = pd.merge(gradu_gu, aca_gu, how='outer', right_index=True, left_index=True)
study_df['평균매매가격'] = pd.to_numeric(price_data.loc['2018-6-1'][local][study_df.index])

In [ ]:
# [예제 4.26] 병합한 데이터프레임으로 scatter 그래프 그리기 

plt.figure(figsize=(12, 6))
plt.scatter(study_df['진학률'], study_df['학원수'], c=study_df['평균매매가격'], s=study_df['평균매매가격']*0.001, cmap="YlOrRd", alpha=0.5)
sns.regplot(study_df['진학률'], study_df['학원수'], scatter=False, color='silver')
plt.xlabel('특목고 진학률(%)')
plt.ylabel('학원수')

for name in study_df.index:
    plt.text(study_df['진학률'][name]*1.01, study_df['학원수'][name], name)
    
plt.colorbar()
plt.show()